In [45]:
#安装目录
install_path='/kaggle/working'

#内网传透
useNgrok = 1 # 非必填 是否使用ngrok作为公网访问地址
#tesoerboard
ngrokTokenFile='/kaggle/input/ngroktoken/Authtoken.txt' # 非必填 存放ngrokToken的文件的路径
port = 6006
#炼丹炉的gui
ngrokTokenFile1='/kaggle/input/ngroktoken/Authtoken1.txt' # 非必填 存放ngrokToken的文件的路径
port1 = 28000

#优化
multi_gpu = 0 
lowram = 0

#删除脚本重新安装
reLoad = 0

#更新脚本
updata = 1

#自动打包训练好的模型
zip_output = 1
#自动清理
clear_output = 1
#自动上传抱脸
upload = 1
hugTokenFile='/kaggle/input/hugfacetoken/hugfacetoken.txt' # 非必填 存放hugfacetoken的文件的路径
#仓库名
repo_id="sukaka/train_model"
#上传仓库路径
path_in_repo="模型.zip"

In [47]:
#开启图形化配置参数
use_gui = 1

#一键倒入炼丹参数
config_file='#/kaggle/input/input-qin/qin.toml'

#炼丹参数配置，如果使用gui或一键导入炼丹参数，请忽略以下
pretrained_model = "/kaggle/input/9527-fp16/9527.safetensors"
is_v2_model = 0
parameterization = 0
train_data_dir = "/kaggle/input/input-qin/input_qin"
reg_data_dir = ""
network_module = "networks.lora"
network_weights = ""
network_dim = 32
network_alpha = 32
resolution = "512,768"
batch_size = 1
max_train_epoches = 10
save_every_n_epochs = 2
train_unet_only = 0
train_text_encoder_only = 0
stop_text_encoder_training = 0
noise_offset = "0"
keep_tokens = 0
min_snr_gamma = 0
lr = "1e-4"
unet_lr = "1e-4"
text_encoder_lr = "1e-5"
lr_scheduler = "cosine_with_restarts"
lr_warmup_steps = 0
lr_restart_cycles = 1
output_name = "qin"
save_model_as = "safetensors"
save_state = 0
resume = ""
min_bucket_reso = 256
max_bucket_reso = 1024
persistent_data_loader_workers = 0
clip_skip = 2

# Optimizer settings
optimizer_type = "AdamW8bit"  # Optimizer type | Available options: AdamW AdamW8bit Lion SGDNesterov SGDNesterov8bit DAdaptation AdaFactor

# LyCORIS training settings
algo = "lora"  # LyCORIS network algo | Available options: lora, loha, lokr, ia3, dylora. lora is equivalent to locon
conv_dim = 4  # conv dim | Similar to network_dim, recommended to be 4
conv_alpha = 4  # conv alpha | Similar to network_alpha, can use same or smaller values than conv_dim
dropout = "0"  # dropout | Dropout probability, 0 means no dropout, larger values mean more dropout. LoHa/LoKr/(IA)^3 do not currently support dropout

In [48]:
import shutil
import gc
import os
import time
import subprocess
import multiprocessing
import inspect
import pkg_resources
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
os.environ['install_path'] = install_path
os.environ['HF_HOME'] = 'huggingface'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["PL_TORCH_DISTRIBUTED_BACKEND"] = "gloo"
  
def libtcmalloc():
    if "LD_PRELOAD" in os.environ and os.environ["LD_PRELOAD"] == "libtcmalloc.so":
        return
    os.chdir('/kaggle')
    os.mkdir('temp')
    os.chdir('temp')
    os.system('wget -qq  http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb')
    os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb')
    os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb')
    os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb')
    os.system('apt install -qq libunwind8-dev -y')
    os.system('dpkg -i *.deb')
    os.system('rm *.deb')
    os.environ["LD_PRELOAD"] = "libtcmalloc.so"
    
def LoRA_scripts_install():
    %cd $install_path
    if reLoad == 1:
        !rm -rf lora-scripts
    if Path("lora-scripts").exists():
        if updata == 1:
            %cd lora-scripts
            !git checkout .
            !git pull
            print('LoRA-scripts已更新')
        else:
            print('LoRA-scripts已安装')
    else:
        print('LoRA-scripts安装中')
        #下载秋叶的脚本
        !git clone --recurse-submodules https://github.com/Akegarasu/lora-scripts
        print('LoRA-scripts已安装')

def venv_install():
    %cd /opt/conda/envs
    if os.path.exists('venv'):
        print('环境已安装')
    else:
        %cd /kaggle/working/
        if not os.path.exists('venv.tar.gz'):
            print('环境包下载中')
            !wget https://huggingface.co/datasets/sukaka/venv_ai_drow/resolve/main/lora_train_py31010_torch2_cu118_xf17.tar.gz -O venv.tar.gz
        print('环境包已下载')
        %cd /opt/conda/envs/
        !mkdir venv
        %cd venv
        print('环境安装中')
        !tar -xzf /kaggle/working/venv.tar.gz
        !source /opt/conda/bin/activate venv
        print('环境安装完毕')
        
def pip():
    installed_packages = [pkg.key for pkg in pkg_resources.working_set]
    if 'huggingface_hub' not in installed_packages or 'pyngrok' not in installed_packages:
        os.system('pip install -q huggingface_hub pyngrok')

In [49]:
def ngrok_start(ngrokTokenFile: str, port: int, address_name: str):
    if Path(ngrokTokenFile).exists():
        with open(ngrokTokenFile, encoding="utf-8") as nkfile:
            ngrokToken = nkfile.readline()
        print('use nrgok')
        from pyngrok import conf, ngrok
        conf.get_default().auth_token = ngrokToken
        conf.get_default().monitor_thread = False
        ssh_tunnels = ngrok.get_tunnels(conf.get_default())
        if len(ssh_tunnels) == 0:
            ssh_tunnel = ngrok.connect(port, bind_tls=True)
            print(f'{address_name}：' + ssh_tunnel.public_url)
        else:
            print(f'{address_name}：' + ssh_tunnels[0].public_url)
    else:
        print('skip start ngrok')

In [50]:
def tensorboard_start():
    ngrok_start(ngrokTokenFile,port,'tensorboard地址')
    %cd $install_path/lora-scripts
    !conda run -n venv --no-capture-output tensorboard --logdir='logs'

def gui(launch_args,port: int):
    import argparse
    import json
    import webbrowser
    from datetime import datetime
    from threading import Lock
    import uvicorn
    from fastapi import BackgroundTasks, FastAPI, Request
    from fastapi.responses import FileResponse
    from fastapi.staticfiles import StaticFiles
    import toml

    app = FastAPI()

    lock = Lock()

    # fix mimetype error in some fucking systems
    sf = StaticFiles(directory="frontend/dist")
    _o_fr = sf.file_response
    def _hooked_file_response(*args, **kwargs):
        full_path = args[0]
        r = _o_fr(*args, **kwargs)
        if full_path.endswith(".js"):
            r.media_type = "application/javascript"
        elif full_path.endswith(".css"):
            r.media_type = "text/css"
        return r
    sf.file_response = _hooked_file_response

    parser = argparse.ArgumentParser(description="GUI for training network")
    parser.add_argument("--port", type=int, default=port, help="Port to run the server on")

    def run_train(toml_path: str):
        print(f"Training started with config file / 训练开始，使用配置文件: {toml_path}")
        cmd = f"conda run -n venv --no-capture-output accelerate launch {' '.join(launch_args)} sd-scripts/train_network.py --config_file {toml_path}"
        try:
            result = subprocess.run(cmd, shell=True, env=os.environ)
            if result.returncode != 0:
                print(f"Training failed / 训练失败")
            else:
                print(f"Training finished / 训练完成")
        except Exception as e:
            print(f"An error occurred when training / 创建训练进程时出现致命错误: {e}")
        finally:
            lock.release()


    @app.post("/api/run")
    async def create_toml_file(request: Request, background_tasks: BackgroundTasks):
        acquired = lock.acquire(blocking=False)

        if not acquired:
            print("Training is already running / 已有正在进行的训练")
            return {"status": "fail", "detail": "Training is already running"}

        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        toml_file = f"toml/{timestamp}.toml"
        toml_data = await request.body()
        j = json.loads(toml_data.decode("utf-8"))
        with open(toml_file, "w") as f:
            f.write(toml.dumps(j))
        background_tasks.add_task(run_train, toml_file)
        return {"status": "success"}

    @app.middleware("http")
    async def add_cache_control_header(request, call_next):
        response = await call_next(request)
        response.headers["Cache-Control"] = "max-age=0"
        return response

    @app.get("/")
    async def index():
        return FileResponse("./frontend/dist/index.html")

    app.mount("/", sf, name="static")

    if __name__ == "__main__":
        args, _ = parser.parse_known_args()
        print(f"Server started at http://127.0.0.1:{port}")

        webbrowser.open(f"http://127.0.0.1:{port}")
        uvicorn.run(app, host="127.0.0.1", port=port, log_level="error")

def train_start():
    %cd /kaggle/working
    !mkdir output
    %cd $install_path/lora-scripts

    ext_args = []
    launch_args = []
    if multi_gpu == 1:
        launch_args.append('--multi_gpu')
        launch_args.append('--num_processes=2')
    if use_gui == 1:
        ngrok_start(ngrokTokenFile1,port1,'炼丹炉gui')
        gui(launch_args,port1)
    else:
        if Path(config_file).is_file():
            cmd = f"""conda run -n venv --no-capture-output accelerate launch {' '.join(launch_args)} "sd-scripts/train_network.py" \
            --config_file={config_file}"""
            os.system(cmd)
        else:
            if lowram == 1:
                ext_args.append('--lowram')

            if is_v2_model == 1:
                ext_args.append('--v2')
            else:
                ext_args.append(f'--clip_skip {clip_skip}')

            if parameterization == 1:
                ext_args.append('--v_parameterization')

            if train_unet_only == 1:
                ext_args.append('--network_train_unet_only')

            if train_text_encoder_only == 1:
                ext_args.append('--network_train_text_encoder_only')

            if network_weights:
                ext_args.append(f'--network_weights {network_weights}')

            if reg_data_dir:
                ext_args.append(f'--reg_data_dir {reg_data_dir}')

            if optimizer_type:
                ext_args.append(f'--optimizer_type {optimizer_type}')

            if optimizer_type == 'DAdaptation':
                ext_args.append('--optimizer_args decouple=True')

            if save_state == 1:
                ext_args.append('--save_state')

            if resume:
                ext_args.append(f'--resume {resume}')

            if persistent_data_loader_workers == 1:
                ext_args.append('--persistent_data_loader_workers')

            if network_module == 'lycoris.kohya':
                ext_args.append(f'--network_args conv_dim={conv_dim} conv_alpha={conv_alpha} algo={algo} dropout={dropout}')

            if stop_text_encoder_training != 0:
                ext_args.append(f'--stop_text_encoder_training {stop_text_encoder_training}')

            if noise_offset != '0':
                ext_args.append(f'--noise_offset {noise_offset}')

            if min_snr_gamma != 0:
                ext_args.append(f'--min_snr_gamma {min_snr_gamma}')

            cmd = f"""conda run -n venv --no-capture-output accelerate launch {' '.join(launch_args)} "sd-scripts/train_network.py" \
            --enable_bucket \
            --pretrained_model_name_or_path={pretrained_model} \
            --train_data_dir={train_data_dir} \
            --output_dir="/kaggle/working/output" \
            --logging_dir="logs" \
            --log_prefix={output_name} \
            --resolution={resolution} \
            --network_module={network_module} \
            --max_train_epochs={max_train_epoches} \
            --learning_rate={lr} \
            --unet_lr={unet_lr} \
            --text_encoder_lr={text_encoder_lr} \
            --lr_scheduler={lr_scheduler} \
            --lr_warmup_steps={lr_warmup_steps} \
            --lr_scheduler_num_cycles={lr_restart_cycles} \
            --network_dim={network_dim} \
            --network_alpha={network_alpha} \
            --output_name={output_name} \
            --train_batch_size={batch_size} \
            --save_every_n_epochs={save_every_n_epochs} \
            --mixed_precision="fp16" \
            --save_precision="fp16" \
            --seed="1337" \
            --cache_latents \
            --prior_loss_weight=1 \
            --max_token_length=225 \
            --caption_extension=".txt" \
            --save_model_as={save_model_as} \
            --min_bucket_reso={min_bucket_reso} \
            --max_bucket_reso={max_bucket_reso} \
            --keep_tokens={keep_tokens} \
            --xformers --shuffle_caption {' '.join(ext_args)} """
            os.system(cmd)


In [ ]:
main():
    startTicks = time.time()
    with ProcessPoolExecutor() as executor:
    futures = [executor.submit(func) for func in [libtcmalloc, pip, LoRA_scripts_install, venv_install]]
    for future in futures:
        future.result()
    startTicks = time.time()
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(func) for func in [tensorboard_start, train_start]]
        for future in futures:
            future.result()

In [51]:
def hugface_upload():
    if Path(hugTokenFile).exists():
        with open(hugTokenFile,encoding = "utf-8") as nkfile:
            hugToken = nkfile.readline()
    if upload == 1 and hugToken != '':
        from huggingface_hub import login
        # 使用您的 Hugging Face 访问令牌登录
        login(token= hugToken)
        from huggingface_hub import HfApi
        # 实例化 HfApi 类
        api = HfApi()
        print("HfApi 类已实例化")

        # 使用 upload_file() 函数上传文件
        print("开始上传文件...")
        response = api.upload_file(
            path_or_fileobj="/kaggle/working/模型.zip",
            path_in_repo={path_in_repo},
            repo_id={repo_id},
            repo_type="dataset"
        )
        print("文件上传完成")
        print(f"响应: {response}")

def zip_clear():
    if zip_output == 1:
        shutil.make_archive('/kaggle/working/模型', 'zip', '/kaggle/working/output/')
        print('模型已压缩到output')
    if clear_output == 1:
        def clear_directory(folder_path):
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
        clear_directory('/kaggle/working/output')
        clear_directory('/kaggle/working/lora-scripts/logs')
        print('清理完毕')

def after_train():
    zip_clear()
    hugface_upload()


In [52]:
main()

/kaggle/working
/opt/conda/envs
环境已安装
/opt/conda/envs/lora-scripts
Updated 0 paths from the index
Already up to date.
LoRA-scripts已更新


加载耗时: 13.430296897888184 s
use nrgok/kaggle/working

tensorboard地址：https://3594-34-27-227-19.ngrok-free.app
/kaggle/working/lora-scripts
mkdir: cannot create directory ‘output’: File exists
/kaggle/working/lora-scripts
use nrgok
炼丹炉gui：https://b292-34-27-227-19.ngrok-free.app
Server started at http://127.0.0.1:28000

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:6006/ (Press CTRL+C to quit)


KeyboardInterrupt: 

^C


In [ ]:
#训练完毕，手动执行。进行打包和上传
after_train()